<a href="https://colab.research.google.com/github/kourtish/deep-rl-class/blob/main/notebooks/bonus-unit1/bonus-unit1_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bonus Unit 1: Let's train Huggy the Dog 🐶 to fetch a stick

In [ ]:
# clone the repository, that contains ML-Agents. (can take 3min)
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

## Setup the Virtual Environment 🔽
- In order for the **ML-Agents** to run successfully in Colab,  Colab's Python version must meet the library's Python requirements.

- We can check for the supported Python version under the `python_requires` parameter in the `setup.py` files. These files are required to set up the **ML-Agents** library for use and can be found in the following locations:
  - `/content/ml-agents/ml-agents/setup.py`
  - `/content/ml-agents/ml-agents-envs/setup.py`

- Colab's Current Python version(can be checked using `!python --version`) doesn't match the library's `python_requires` parameter, as a result installation may silently fail and lead to errors like these, when executing the same commands later:
  - `/bin/bash: line 1: mlagents-learn: command not found`
  - `/bin/bash: line 1: mlagents-push-to-hf: command not found`

- To resolve this, we'll create a virtual environment with a Python version compatible with the **ML-Agents** library.

`Note:` *For future compatibility, always check the `python_requires` parameter in the installation files and set your virtual environment to the maximum supported Python version in the given below script if the Colab's Python version is not compatible*

In [ ]:
# I check this files, this lib require python_requires=">=3.10.1,<=3.10.12"
# Colab's Current Python Version (Incompatible with ML-Agents)
!python --version

In [ ]:
%%capture
# Install virtualenv and create a virtual environment
!pip install virtualenv
!virtualenv myenv

# Download and install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Activate Miniconda and install Python ver 3.10.12
!source /usr/local/bin/activate
!conda install -q -y --prefix /usr/local python=3.10.12 ujson  # Specify the version here

# Set environment variables for Python and conda paths
!export PYTHONPATH=/usr/local/lib/python3.10/site-packages/
!export CONDA_PREFIX=/usr/local/envs/myenv

In [ ]:
# Python Version in New Virtual Environment (Compatible with ML-Agents)
!python --version

## Installing the dependencies 🔽

In [ ]:
%%capture
# Go inside the repository and install the package (can take 3min)
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

## Download and move the environment zip file in `./trained-envs-executables/linux/`

- Our environment executable is in a zip file.
- We need to download it and place it to `./trained-envs-executables/linux/`

In [ ]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

In [ ]:
!wget "https://github.com/huggingface/Huggy/raw/main/Huggy.zip" -O ./trained-envs-executables/linux/Huggy.zip

In [ ]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

In [ ]:
!chmod -R 755 ./trained-envs-executables/linux/Huggy # Make sure your file is accessible

In [ ]:
!cd /content/ml-agents/config/ppo && ls

In [ ]:
content = """behaviors:
  Huggy:
    trainer_type: ppo
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: true
      hidden_units: 512
      num_layers: 3
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.995
        strength: 1.0
    checkpoint_interval: 200000
    keep_checkpoints: 15
    max_steps: 2e6
    time_horizon: 1000
    summary_freq: 50000"""

import os

file_path = "/content/ml-agents/config/ppo/Huggy.yaml"
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, 'w') as f:
    f.write(content)

print(f"Content successfully written to {file_path}")

In [ ]:
# The training will take 30 to 45min
!mlagents-learn ./config/ppo/Huggy.yaml --env=./trained-envs-executables/linux/Huggy/Huggy --run-id="Huggy2" --no-graphics

In [ ]:
# Push the agent to the 🤗 Hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# mlagents-push-to-hf
!mlagents-push-to-hf --run-id="HuggyTraining" --local-dir="./results/Huggy2" --repo-id="g-kurtish/ppo-Huggy" --commit-message="Huggy"

## Play with your Huggy 🐕

This step is the simplest:

- Open the game Huggy in your browser: https://huggingface.co/spaces/ThomasSimonini/Huggy